In [173]:
import os

In [174]:
import warnings
warnings.filterwarnings("ignore") 

In [175]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGSMITH_TRACING=os.getenv("LANGSMITH_TRACING_V2", "true")  # Default to "true" if not set
LANGSMITH_ENDPOINT=os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")


os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT

### Load the model and test it with a simple message

In [177]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [178]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    convert_system_message_to_human=True,
)

In [179]:
message1 = [SystemMessage(content="You are a helpful assistant."),
           HumanMessage(content="I need your help")]

In [180]:
response1 = llm.invoke(message1)
response1

AIMessage(content="Okay, I'm ready to help! Tell me what you need. The more information you give me, the better I can assist you. For example, you could tell me:\n\n*   **What you're trying to achieve:** What is your goal?\n*   **What you've already tried:** What steps have you taken so far?\n*   **What problems you're encountering:** What's blocking you?\n*   **What kind of help you need:** Are you looking for information, code, creative writing, brainstorming, or something else?\n\nI can help with a wide range of tasks, including:\n\n*   Answering questions\n*   Generating text (stories, poems, code, etc.)\n*   Summarizing information\n*   Translating languages\n*   Brainstorming ideas\n*   Providing definitions\n*   Offering advice (though I'm not a substitute for professional advice)\n\nI'm looking forward to hearing from you! Let me know how I can help.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP',

In [181]:
message2 = [SystemMessage(content="You are a helpful assistant. and you give concise answers."),
           HumanMessage(content="I need your help")]

In [182]:
response2 = llm.invoke(message2)
response2

AIMessage(content="Okay, I'm ready. How can I help you?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--74b56e9d-d65c-43a8-97c4-c23b3819ba40-0', usage_metadata={'input_tokens': 16, 'output_tokens': 14, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}})

### Using Output Perser

In [183]:
from langchain_core.output_parsers import StrOutputParser

In [184]:
parser = StrOutputParser()
parser.invoke(response1)

"Okay, I'm ready to help! Tell me what you need. The more information you give me, the better I can assist you. For example, you could tell me:\n\n*   **What you're trying to achieve:** What is your goal?\n*   **What you've already tried:** What steps have you taken so far?\n*   **What problems you're encountering:** What's blocking you?\n*   **What kind of help you need:** Are you looking for information, code, creative writing, brainstorming, or something else?\n\nI can help with a wide range of tasks, including:\n\n*   Answering questions\n*   Generating text (stories, poems, code, etc.)\n*   Summarizing information\n*   Translating languages\n*   Brainstorming ideas\n*   Providing definitions\n*   Offering advice (though I'm not a substitute for professional advice)\n\nI'm looking forward to hearing from you! Let me know how I can help."

In [185]:
parser.invoke(response2)

"Okay, I'm ready. How can I help you?"

### Chaining with LCEL
- LangChain Expression Language (LCEL) &mdash; a concise syntax for composing and chaining together components (like models and output parsers) in LangChain workflows.LCEL

In [186]:
chain = llm | parser

- The line `chain = llm | parser` creates a simple pipeline (chain) where the output of the language model (`llm`) is passed directly to the output parser (`parser`). This allows you to process a prompt through the model and automatically parse its response in a single step, making it easier to work with structured outputs in LangChain.

In [187]:
chain.invoke(message1)

"Okay, I'm ready to help! Tell me what you need. The more information you give me, the better I can assist you. For example, you could tell me:\n\n*   **What you're trying to achieve:** What is your goal?\n*   **What you've already tried:** What steps have you taken so far?\n*   **What problems you're encountering:** What's blocking you?\n*   **What kind of help you need:** Are you looking for information, code, creative writing, brainstorming, or something else?\n\nI can help with a wide range of tasks, including:\n\n*   Answering questions\n*   Generating text (stories, poems, code, etc.)\n*   Summarizing information\n*   Translating languages\n*   Brainstorming ideas\n*   Providing definitions\n*   Offering advice (though I'm not a substitute for professional advice)\n\nI'm looking forward to hearing from you! Let me know how I can help."

In [188]:
chain.invoke(message2)

"Okay, I'm ready. How can I help you?"

### Prompting with PromptTemplate

In [189]:
from langchain_core.prompts import ChatPromptTemplate

In [190]:
promt_template = ChatPromptTemplate.from_messages(
    [("system", "You give concise answer and Translate the following text to : {language}"),
     ("human", "{text}")]
)
    

In [191]:
prompt = promt_template.invoke({"language": "Bengali", "text": "My name is Durjay."})

In [192]:
prompt.to_messages()

[SystemMessage(content='You give concise answer and Translate the following text to : Bengali', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Durjay.', additional_kwargs={}, response_metadata={})]

In [193]:
response = llm.invoke(prompt)

In [194]:
response.content

'আমার নাম Durjay। (Amar naam Durjay.)'

### Chaining

In [195]:
chain = promt_template | llm | parser

In [196]:
chain.invoke({"language": "Bengali", "text": "My name is Durjay."})

'আমার নাম Durjay। (Amar naam Durjay.)'